In [ ]:
def xpath2xml(xpath, xmlns='', remove=False):
    #transforms xpath like string e.g. "/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni=10444"
    #to xml-like sequence of elements tags: 
    # <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    #   <eps-items>
    #       <epId-items>
    #           <Ep-list>
    #               <epId>1</epId>
    #               <nws-items>
    #                   <vni-items>
    #                       <Nw-list operation="remove">
    #                           <vni>10444</vni>
    #                       </Nw-list>
    #                   </vni-items>
    #               </nws-items>
    #           </Ep-list>
    #       </epId-items>
    #   </eps-items>
    # </System>
    #
    # it has optional parameter 'remove' that adds the string 'operation="remove"' to the element marked with square brackets '[]'

    pl = xpath.split('/')
    
    xmls = f'<{pl[1]} xmlns="{xmlns}">' if xmlns else f'<{pl[1]}>'
    xmle = f'</{pl[1]}>'
     
    def _xpath2xml(pl):
        key = ''
        xmls = ''
        xmle = ''

        for i in range(len(pl)):
            elem = pl[i]
            if "=" in elem:
                elem,key = elem.split("=")
                xmls += f'<{elem}>{key}</{elem}>'
                break
            if "[]" in elem:
                elem = elem[:-2]
                if remove:
                    xmls += f'<{elem} operation="remove">'                
                else:
                    xmls += f'<{elem}>'
            else:                    
                xmls += f'<{elem}>'
            xmle = f'</{elem}>' + xmle
     
        if key and i < len(pl)-1:
            return xmls + _xpath2xml(pl[(i+1)::]) + xmle #recursion
        else:
            return xmls + xmle
     
    return xmls + _xpath2xml(pl[2::]) + xmle

In [ ]:
from scrapli_netconf.driver import NetconfScrape
my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,    
}

conn = NetconfScrape(**my_device)
conn.open()
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"

In [ ]:
vlan_id = 222
vxlan_id = 10222

In [ ]:
# get all evpn related configs by netconf

xpath_nve = f"/System/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}"
xpath_svi = f"/System/intf-items/svi-items/If-list[]/id=vlan{vlan_id}"
xpath_vlan = f"/System/bd-items/bd-items/BD-list/fabEncap=vlan-{vlan_id}"
xpath_evpn = f"/System/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}"

rpc_nve = xpath2xml(xpath_nve, xmlns)
rpc_svi = xpath2xml(xpath_svi, xmlns)
rpc_vlan = xpath2xml(xpath_vlan, xmlns)
rpc_evpn = xpath2xml(xpath_evpn, xmlns)

response = conn.get(filter_=rpc_nve, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_svi, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_vlan, filter_type="subtree")
print(response.result)
response = conn.get(filter_=rpc_evpn, filter_type="subtree")
print(response.result)

In [ ]:
# delete given evpn

del_evpn_conf = ('<config><System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">' + 
        xpath2xml(f"/eps-items/epId-items/Ep-list/epId=1/nws-items/vni-items/Nw-list[]/vni={vxlan_id}", '', remove=True) + 
        xpath2xml(f"/intf-items/svi-items/If-list[]/id=vlan{vlan_id}", '', remove=True) + 
        xpath2xml(f"/bd-items/bd-items/BD-list[]/fabEncap=vlan-{vlan_id}", '', remove=True) + 
        xpath2xml(f"/evpn-items/bdevi-items/BDEvi-list[]/encap=vxlan-{vxlan_id}", '', remove=True) + 
"</System></config>")
print(conf)

In [ ]:
response = conn.edit_config(config=del_evpn_conf, target="running")
print(response.result)

In [ ]:
conn.close()